# EDA

In [1]:
import numpy as np
import pandas as pd

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("mov_rec") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
rat_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/ratings.csv', header=True)
        

In [4]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [51]:
rat_rdd.tail(5)

[Row(userId=610, movieId=166534, rating=4.0, timestamp=1493848402),
 Row(userId=610, movieId=168248, rating=5.0, timestamp=1493850091),
 Row(userId=610, movieId=168250, rating=5.0, timestamp=1494273047),
 Row(userId=610, movieId=168252, rating=5.0, timestamp=1493846352),
 Row(userId=610, movieId=170875, rating=3.0, timestamp=1493846415)]

In [5]:
rat_rdd.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [6]:
mov_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/movies.csv', header=True)
        

In [7]:
# tags_rdd = spark.read.format('com.databricks.spark.csv').\
#                              options(header='true',\
#                              inferschema='true').\
#                              load('data/ml-latest-small/tags.csv', header=True)
        

In [8]:
mov_rdd.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [9]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [10]:
mov_rdd.createOrReplaceTempView("mov_rdd")
rat_rdd.createOrReplaceTempView("rat_rdd")

In [11]:
# tags_rdd.show(5)

Leave out timestamp...

In [23]:
mov_surprise = spark.sql(
'''
SELECT ratings.userId, movies.title, ratings.rating 
  FROM mov_rdd AS movies
  JOIN rat_rdd AS ratings
    ON movies.movieId = ratings.movieId
'''
)

In [24]:
mov_surprise.createOrReplaceTempView("mov_surprise")

In [25]:
mov_surprise.show(5)

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|     1|    Toy Story (1995)|   4.0|
|     1|Grumpier Old Men ...|   4.0|
|     1|         Heat (1995)|   4.0|
|     1|Seven (a.k.a. Se7...|   5.0|
|     1|Usual Suspects, T...|   5.0|
+------+--------------------+------+
only showing top 5 rows



# Surprise

Begin by reimplementing the similarity based recommender from lecture

In [18]:
pip install surprise

     |████████████████████████████████| 11.8 MB 18.0 MB/s eta 0:00:01   |████████                        | 3.0 MB 2.2 MB/s eta 0:00:05
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-linux_x86_64.whl size=2325425 sha256=459c5b53fbd9527e2f79f3e5a57881a5d6c975f92ffe23adb57432ecdf7c3cc6
  Stored in directory: /home/jovyan/.cache/pip/wheels/20/91/57/2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [42]:
from surprise import KNNBasic, KNNWithMeans, SVD

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate

mov_surprise = mov_surprise[mov_surprise['rating']!=0]

reader = Reader(rating_scale=(1,5))

In [43]:
mov_surprise.head(5)

[Row(userId=1, title='Toy Story (1995)', rating=4.0),
 Row(userId=1, title='Grumpier Old Men (1995)', rating=4.0),
 Row(userId=1, title='Heat (1995)', rating=4.0),
 Row(userId=1, title='Seven (a.k.a. Se7en) (1995)', rating=5.0),
 Row(userId=1, title='Usual Suspects, The (1995)', rating=5.0)]

In [44]:
# convert to pandas DataFrame
# warning, if file is too large, this will likely crash...
mov_pDF = mov_surprise.toPandas()
print(mov_pDF.head(5))

surprise_movie_ratings = Dataset.load_from_df(mov_pDF, reader)

   userId                        title  rating
0       1             Toy Story (1995)     4.0
1       1      Grumpier Old Men (1995)     4.0
2       1                  Heat (1995)     4.0
3       1  Seven (a.k.a. Se7en) (1995)     5.0
4       1   Usual Suspects, The (1995)     5.0


In [45]:
# treat data as training set
training_movie_data = surprise_movie_ratings.build_full_trainset()

# anything not a user-item rating pair as test
# no data for this, so we can't do an eval on out of sample accuracy
testing_movie_data = training_movie_data.build_anti_testset()

# create instance of recommender algorithm object
algo = KNNBasic()

# fit it on the training data
algo.fit(training_movie_data)

# predict on the test data
predictions = algo.test(testing_movie_data)

# print the first 5 predictions
predictions[:5]

Computing the msd similarity matrix...
Done computing similarity matrix.


[Prediction(uid=1, iid='Shawshank Redemption, The (1994)', r_ui=3.501556983616962, est=4.7933170341863915, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid='Good Will Hunting (1997)', r_ui=3.501556983616962, est=4.406962278623285, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid='Kill Bill: Vol. 1 (2003)', r_ui=3.501556983616962, est=4.232475482922819, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid='Collateral (2004)', r_ui=3.501556983616962, est=3.86890093822762, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid='Talladega Nights: The Ballad of Ricky Bobby (2006)', r_ui=3.501556983616962, est=3.3081937462915807, details={'actual_k': 28, 'was_impossible': False})]

In [19]:
type(mov_surprise)

pyspark.sql.dataframe.DataFrame

In [46]:
# standard format for cross validation
# algorithm used, ratings dataset, metrics, number of cross validations, verbose controls how much output
cross_validate(SVD(n_factors=5, reg_all=0.005), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8716  0.8661  0.8637  0.8699  0.8750  0.8693  0.0040  
MAE (testset)     0.6690  0.6684  0.6635  0.6687  0.6714  0.6682  0.0026  
Fit time          4.03    3.91    3.49    3.30    3.30    3.61    0.31    
Test time         0.45    0.39    0.39    0.32    0.38    0.39    0.04    


{'test_rmse': array([0.87164105, 0.86614005, 0.86367178, 0.86992878, 0.87500647]),
 'test_mae': array([0.6689906 , 0.66837099, 0.66353079, 0.66872449, 0.6713929 ]),
 'fit_time': (4.034169435501099,
  3.9060375690460205,
  3.4865376949310303,
  3.3026998043060303,
  3.299785852432251),
 'test_time': (0.45351123809814453,
  0.3940465450286865,
  0.3924839496612549,
  0.3172287940979004,
  0.3789184093475342)}

In [47]:
cross_validate(KNNBasic(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9617  0.9497  0.9607  0.9503  0.9603  0.9565  0.0054  
MAE (testset)     0.7297  0.7255  0.7328  0.7240  0.7313  0.7287  0.0034  
Fit time          0.15    0.26    0.19    0.26    0.20    0.21    0.04    
Test time         2.75    2.89    2.71    2.71    2.84    2.78    0.07    


{'test_rmse': array([0.96169401, 0.94967837, 0.96065888, 0.9502791 , 0.9602762 ]),
 'test_mae': array([0.72974103, 0.72547832, 0.73277338, 0.72398267, 0.73134909]),
 'fit_time': (0.1521005630493164,
  0.2565803527832031,
  0.19399213790893555,
  0.26407527923583984,
  0.20015239715576172),
 'test_time': (2.7461354732513428,
  2.8907337188720703,
  2.714517831802368,
  2.7065658569335938,
  2.839329242706299)}

In [ ]:
cross_validate(KNNWithMeans(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)